[View in Colaboratory](https://colab.research.google.com/github/m-toman/notebooks/blob/master/Adorno_v1.ipynb)

**Install**

In [1]:
# install CUDA etc.
#!wget https://developer.nvidia.com/compute/cuda/9.2/Prod2/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.148-1_amd64
#!dpkg --install cuda-repo-ubuntu1604-9-2-local_9.2.148-1_amd64
#!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
#!apt-get update
#!apt-get install cuda
#!cat /usr/local/cuda/version.txt
#!pip install http://download.pytorch.org/whl/cu92/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
#!pip install torchvision  

! apt-get install portaudio19-dev
! pip install pyaudio falcon inflect audioread librosa matplotlib tqdm Unidecode sounddevice lws torch

import os
from os.path import exists, expanduser

tacorn_dir = os.path.join(expanduser("~"), "tacorn")
tacotron2_dir = os.path.join(tacorn_dir, "tacotron2")

os.chdir(expanduser("~"))
! rm -rf $tacorn_dir
if not exists(tacorn_dir):
  ! git clone https://github.com/m-toman/tacorn.git
    
os.chdir(tacorn_dir)
! git clone https://github.com/m-toman/Tacotron-2 tacotron2
! touch tacotron2/__init__.py

# get wavernn model
os.chdir(tacorn_dir)
! mkdir -p model_checkpoints
os.chdir(os.path.join(tacorn_dir, "model_checkpoints"))
! curl -O -L https://www.dropbox.com/s/06erqg65b44xtb3/model_adorno.zip
! unzip model_adorno.zip
! rm model_adorno.zip

# get tacotron2 model
os.chdir(tacotron2_dir)
! curl -O -L https://www.dropbox.com/s/4ucqswejkk61cea/logs-Tacotron-2.zip
! unzip logs-Tacotron-2.zip
! rm logs-Tacotron-2.zip
! rm hparams.py
! curl -O -L https://www.dropbox.com/s/kqmob7pe8lsmxzz/hparams.py

os.chdir(tacorn_dir)
os.chdir(tacotron2_dir)


Reading package lists... Done
Building dependency tree       
Reading state information... Done
portaudio19-dev is already the newest version (19.6.0-1).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.
Cloning into 'tacorn'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 113 (delta 10), reused 19 (delta 10), pack-reused 90
Receiving objects: 100% (113/113), 36.14 KiB | 12.05 MiB/s, done.
Resolving deltas: 100% (57/57), done.
Cloning into 'tacotron2'...
remote: Enumerating objects: 983, done.
remote: Total 983 (delta 0), reused 0 (delta 0), pack-reused 983
Receiving objects: 100% (983/983), 9.12 MiB | 12.13 MiB/s, done.
Resolving deltas: 100% (640/640), done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- -

**Synthesize (Griffin & Lim)**

In [15]:
texts = [
    "Dies ist eine großartige Leistung, wohlgeformt und elegant.",
    "Langeweile ist der Reflex auf das objektive Grau. Langeweile ist objektive Verzweiflung."
]

os.chdir(tacotron2_dir)
with open("text_list.txt", "wt") as fp:
    for l in texts:
        fp.write(l + "\n")

os.chdir(tacotron2_dir)
! rm -rf tacotron_output
! python synthesize.py --model Tacotron --tacotron_name Tacotron-2 --mode eval --text_list text_list.txt &> /dev/null
#! ls tacotron_output/logs-eval/wavs
import IPython
from IPython.display import Audio

wavsdir = "tacotron_output/logs-eval/wavs"
wavs = [f for f in os.listdir(wavsdir) if f.endswith("linear.wav")]

for wav in wavs:
    IPython.display.display(Audio(filename=os.path.join(wavsdir, wav)))


**Synthesize 2 (WaveRNN)**

In [12]:
texts = [
    "Dies ist eine großartige Leistung, wohlgeformt und elegant.",
    "angeweile ist der Reflex auf das objektive Grau. Langeweile ist objektive Verzweiflung."
]

os.chdir(tacorn_dir)
with open("text_list.txt", "wt") as fp:
    for l in texts:
        fp.write(l + "\n")

os.chdir(tacorn_dir)
! rm -rf synthesized_wavs
! python synthesize.py --sentences_file text_list.txt

import IPython
from IPython.display import Audio

wavsdir = "synthesized_wavs"
wavs = [f for f in os.listdir(wavsdir) if f.endswith(".wav")]

for wav in wavs:
    IPython.display.display(Audio(filename=os.path.join(wavsdir, wav)))


Using TensorFlow backend.
loaded model at tacotron2/logs-Tacotron-2/taco_pretrained/tacotron_model.ckpt-60000
Hyperparameters:
  allow_clipping_in_normalization: True
  attention_dim: 128
  attention_filters: 32
  attention_kernel: (31,)
  cbhg_conv_channels: 128
  cbhg_highway_units: 128
  cbhg_highwaynet_layers: 4
  cbhg_kernels: 8
  cbhg_pool_size: 2
  cbhg_projection: 256
  cbhg_projection_kernel_size: 3
  cbhg_rnn_units: 128
  cin_channels: 80
  cleaners: transliteration_cleaners
  clip_for_wavenet: True
  clip_mels_length: True
  cross_entropy_pos_weight: 20
  cumulative_weights: True
  decoder_layers: 2
  decoder_lstm_units: 1024
  embedding_dim: 512
  enc_conv_channels: 512
  enc_conv_kernel_size: (5,)
  enc_conv_num_layers: 3
  encoder_lstm_units: 256
  fmax: 7600
  fmin: 55
  frame_shift_ms: None
  freq_axis_kernel_size: 3
  gate_channels: 256
  gin_channels: -1
  griffin_lim_iters: 60
  hop_size: 275
  input_type: raw
  kernel_size: 3
  layers: 20
  leaky_alpha: 0.4
  log_sc

FileNotFoundError: ignored